### Consolidate Yloader OHLCV Data

This notebook reads all individual ticker `.csv` files generated by the `Yloader` application, combines them into a single, analysis-ready DataFrame, and saves it in the efficient Parquet format.

**Workflow:**

1.  **Prerequisite:** Run `Yloader` to download OHLCV data. This should create multiple `.csv` files (e.g., `AAPL.csv`, `GOOG.csv`) in a dedicated directory.
2.  **Process & Combine:** The notebook scans the source directory, reads each CSV, and consolidates them into a single pandas DataFrame with a `(Ticker, Date)` MultiIndex.
3.  **Save:** The final DataFrame is saved as a single `.parquet` file for fast loading in subsequent analysis notebooks.
4.  **Verify:** The saved Parquet file is read back to confirm its integrity.

### Setup and Configuration

**This is the only cell you need to modify.** Adjust the paths and column definitions below to match your project structure.

In [1]:
import sys
import pandas as pd
from pathlib import Path
from typing import List, Optional

# --- Configuration ---

# 1. Set the directory where your Yloader CSV files are located.
#    This uses Path.home() to be portable across different computers and OS.
YLOADER_DATA_DIR = Path.home() / "Desktop" / "yloader"

# 2. Define the destination path for the final combined Parquet file.
#    It's good practice to save data outputs within the project's data folder.
#    Assuming a project structure where this notebook is in a `notebooks` subdir.
# ROOT_DIR = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
# DESTINATION_PARQUET_PATH = ROOT_DIR / "data" / "df_OHLCV_stocks_etfs.parquet"
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent 
SRC_DIR = ROOT_DIR / 'src'
if str(ROOT_DIR) not in sys.path: sys.path.append(str(ROOT_DIR))
if str(SRC_DIR) not in sys.path: sys.path.append(str(SRC_DIR))
DESTINATION_PARQUET_PATH = ROOT_DIR / "data" / "df_OHLCV_stocks_etfs.parquet"

# 3. Define the column names for the CSV files.
#    Assumes CSVs have no header and columns are in this fixed order.
#    The first column is always assumed to be 'Date'.
CANONICAL_COLUMN_NAMES = ['Adj Open', 'Adj High', 'Adj Low', 'Adj Close', 'Volume']

# --- Verification ---
print(f"Project Root Directory set to: {ROOT_DIR}")
print(f"Reading Yloader CSVs from: {YLOADER_DATA_DIR}")
print(f"Output will be saved to: {DESTINATION_PARQUET_PATH}")

# Set pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

Project Root Directory set to: c:\Users\ping\Files_win10\python\py311\stocks
Reading Yloader CSVs from: C:\Users\ping\Desktop\yloader
Output will be saved to: c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet


### Step 1: Process and Combine CSV Data

This cell defines and executes the core logic to read all individual CSV files and merge them into a single, multi-indexed DataFrame.

In [2]:
def process_yloader_csvs(
    data_dir: Path,
    canonical_cols: List[str]
) -> Optional[pd.DataFrame]:
    """
    Reads and combines all CSV files from the Yloader output directory.

    Args:
        data_dir (Path): Path object pointing to the directory with CSV files.
        canonical_cols (List[str]): Expected data column names, excluding 'Date'.

    Returns:
        Optional[pd.DataFrame]: A sorted, multi-indexed DataFrame, or None if no
                                data could be processed.
    """
    if not data_dir.is_dir():
        print(f"Error: Directory not found: {data_dir}")
        return None

    csv_files = list(data_dir.glob('*.csv'))
    if not csv_files:
        print(f"No CSV files found in directory: {data_dir}")
        return None

    print(f"Found {len(csv_files)} CSV files to process...")
    all_dataframes = []
    tickers_list = []
    
    # The full list of column names, including the 'Date' index column
    expected_csv_cols = ['Date'] + canonical_cols

    for file_path in csv_files:
        # Use pathlib's `.stem` to get the filename without the extension (e.g., "AAPL.csv" -> "AAPL")
        ticker = file_path.stem
        try:
            df_temp = pd.read_csv(
                file_path,
                header=None,
                names=expected_csv_cols,
                parse_dates=['Date'],
                index_col='Date'
            )
            
            if df_temp.empty:
                print(f"Warning: File {file_path.name} is empty. Skipping.")
                continue

            all_dataframes.append(df_temp)
            tickers_list.append(ticker)

        except Exception as e:
            print(f"An error occurred while processing {file_path.name}: {e}. Skipping.")

    if not all_dataframes:
        print("No data was successfully loaded. Aborting.")
        return None

    # Concatenate all DataFrames with a 'Ticker' key in the new index
    multi_index_df = pd.concat(all_dataframes, keys=tickers_list, names=['Ticker', 'Date'])
    
    # Sort by Ticker (A-Z), then by Date (newest first)
    multi_index_df.sort_index(level=['Ticker', 'Date'], ascending=[True, False], inplace=True)
    
    print("\nCSV processing complete. DataFrame created successfully.")
    return multi_index_df

# --- Execute Step 1 ---
final_df = process_yloader_csvs(YLOADER_DATA_DIR, CANONICAL_COLUMN_NAMES)

if final_df is not None:
    print("\n--- DataFrame Info ---")
    final_df.info()
    
    print("\n--- First 5 Rows of Combined DataFrame ---")
    display(final_df.head())

Found 1590 CSV files to process...

CSV processing complete. DataFrame created successfully.

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 998303 entries, ('A', Timestamp('2025-07-25 00:00:00')) to ('ZWS', Timestamp('2023-01-03 00:00:00'))
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Open   998303 non-null  float64
 1   Adj High   998303 non-null  float64
 2   Adj Low    998303 non-null  float64
 3   Adj Close  998303 non-null  float64
 4   Volume     998303 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 41.9+ MB

--- First 5 Rows of Combined DataFrame ---


Adj Open  Adj High  Adj Low  Adj Close   Volume
Ticker Date                                                       
A      2025-07-25    120.50    121.01   119.02     120.18  1972700
       2025-07-24    119.43    122.75   119.43     120.35  1602800
       2025-07-23    119.38    120.79   117.48     119.47  1939700
       2025-07-22    112.85    117.05   112.14     116.19  1692000
       2025-07-21    113.23    113.66   110.75     112.04  2060800

### Step 2: Save DataFrame to Parquet File

This step saves the combined DataFrame to a Parquet file, which is highly efficient for storage and subsequent loading.


In [3]:
def save_dataframe_to_parquet(df: Optional[pd.DataFrame], dest_path: Path):
    """
    Saves a DataFrame to a Parquet file if it's not empty.

    Args:
        df (Optional[pd.DataFrame]): The DataFrame to save.
        dest_path (Path): The destination file path.
    """
    if df is None or df.empty:
        print("DataFrame is empty or None. Nothing to save.")
        return

    try:
        # Ensure the destination directory exists
        dest_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Save the file using the efficient 'pyarrow' engine and 'zstd' compression
        df.to_parquet(dest_path, engine='pyarrow', compression='zstd', index=True)
        print(f"\nSuccessfully saved DataFrame to: {dest_path}")
        
    except Exception as e:
        print(f"\nError: Failed to save Parquet file. Details: {e}")

# --- Execute Step 2 ---
save_dataframe_to_parquet(final_df, DESTINATION_PARQUET_PATH)


Successfully saved DataFrame to: c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet


### Step 3: Verify the Saved File (Optional)

This final step reads the Parquet file back into a new DataFrame to ensure the data was saved correctly.

In [4]:
print(f"--- Verifying the saved file at: {DESTINATION_PARQUET_PATH} ---")

try:
    if DESTINATION_PARQUET_PATH.exists():
        verified_df = pd.read_parquet(DESTINATION_PARQUET_PATH)
        print("Verification successful! File read back into memory.")
        
        print("\n--- First 5 Rows of Verified DataFrame ---")
        display(verified_df.head())
        
        # Optional: Check a specific ticker
        if not verified_df.empty:
            example_ticker = verified_df.index.get_level_values('Ticker')[0]
            print(f"\n--- Data for first available ticker '{example_ticker}' ---")
            display(verified_df.loc[example_ticker].head())

    else:
        print("Error: The output file was not found at the specified path.")
        
except Exception as e:
    print(f"An error occurred during verification: {e}")

--- Verifying the saved file at: c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet ---
Verification successful! File read back into memory.

--- First 5 Rows of Verified DataFrame ---


Adj Open  Adj High  Adj Low  Adj Close   Volume
Ticker Date                                                       
A      2025-07-25    120.50    121.01   119.02     120.18  1972700
       2025-07-24    119.43    122.75   119.43     120.35  1602800
       2025-07-23    119.38    120.79   117.48     119.47  1939700
       2025-07-22    112.85    117.05   112.14     116.19  1692000
       2025-07-21    113.23    113.66   110.75     112.04  2060800


--- Data for first available ticker 'A' ---


,Adj Open,Adj High,Adj Low,Adj Close,Volume
Date,,,,,
2025-07-25,120.50,121.01,119.02,120.18,1972700
2025-07-24,119.43,122.75,119.43,120.35,1602800
2025-07-23,119.38,120.79,117.48,119.47,1939700
2025-07-22,112.85,117.05,112.14,116.19,1692000
2025-07-21,113.23,113.66,110.75,112.04,2060800
